# Global Setup

In [252]:
!pip install transformers==4.5.0
!pip install sentencepiece
!pip install segtok
!pip install vaderSentiment
!pip install nltk
!pip install huggingface_hub

In [253]:
import os
import sys
from pathlib import Path

import json
import pandas as pd
import random

import torch
from segtok import tokenizer
from keras.preprocessing.sequence import pad_sequences
import tqdm

from multiprocessing import Pool

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import tokenize

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [254]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [255]:
ROOT_FOLDER = Path("/content/drive/My Drive/cs182_final_project/cs182-nlp")
DATA_FOLDER = ROOT_FOLDER / "dataset"
TORCH_CHECKPOINT_MODEL = ROOT_FOLDER / "output" / "training_checkpoint_chandana.pt"

input("Please check to make sure the above checkpoint directory is yours (Hit any key)")

Please check to make sure the above checkpoint directory is yours (Hit any key)


''

In [256]:
sys.path.append(ROOT_FOLDER)

In [257]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [258]:
list_to_device = lambda th_obj: [tensor.to(device) for tensor in th_obj]

# Model Params

In [259]:
MAX_LEN = 128
#MAX_LEN_VADER = 40
BATCH_SIZE = 32
EPOCHS = 5

# Higher bound settings: MAX_LEN = 256 and BATCH_SIZE = 16

#Data Preprocessing Functions

## load data

In [260]:
def load_json(file_path, filter_function=lambda x: True):
  """
  file_path - full path of the file to read from
  filter_function - a data selection function, returns True to ADD a data point
  """
  result = []

  try:
    with open(file_path, "r") as f:
      for line in f:
        json_line = json.loads(line)
        if not filter_function(json_line):
          # Disallow via opposite of allow
          continue
        result.append(json_line) # each line is one data point dictionary
    return pd.DataFrame.from_records(result)
    # return result

  except IOError:
    print(f"cannot open {file_path}")
    return None

## data formatting

### tokenize

In [261]:
def tokenize(data):
  """
  data - an iterable of sentences
  """
  token_set = set()
  i = 0
  for sentences in data:
    if i % 1000 == 0:
      print(i, end=", " if i % 15000 != 0 else "\n")
    tokenized = nltk.word_tokenize(sentences.lower())
    for token in tokenized:
      token_set.add(token)
    i += 1
  return token_set

In [262]:
def tokenize_review(tokenizer, review_text):
  encodings = tokenizer.encode_plus(review_text, add_special_tokens=True,
                                    max_length=MAX_LEN,
                                    return_token_type_ids=False,
                                    return_attention_mask=False,
                                    truncation=True,
                                    pad_to_max_length=False)
  return encodings.get("input_ids", [])


### padding

In [263]:
def pad_sequence(numerized, pad_index, to_length, beginning=True):
    pad = numerized[:to_length]
    if beginning:
      padded = [pad_index] * (to_length - len(pad)) + pad
    else:
      padded = pad + [pad_index] * (to_length - len(pad))
    mask = [w != pad_index for w in padded]
    return padded, mask

### batching

In [264]:
def batch_to_torch_long(*batches):
  if len(batches) == 1:
    return torch.LongTensor(batches[0])
  return [torch.LongTensor(batch) for batch in batches]

def batch_to_torch_float(*batches):
  if len(batches) == 1:
    return torch.FloatTensor(batches[0])
  return [torch.FloatTensor(batch) for batch in batches]

### full data format

### split dataset

In [265]:
# https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=0):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    # m = df.size
    m = len(df.index)

    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end

    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]

    assert train.size + validate.size + test.size == df.size

    return train, validate, test

# Data Preprocessing Code

## load data

In [266]:
# load yelp data
yelp_full = load_json(DATA_FOLDER / "yelp_review_training_dataset.jsonl")
print("loaded", len(yelp_full.index), "data points")

loaded 533581 data points


In [347]:
yelp_reviews = yelp_full[0:30000]
print(yelp_reviews['stars'].to_list())

[1.0, 5.0, 5.0, 5.0, 1.0, 4.0, 3.0, 1.0, 2.0, 3.0, 4.0, 1.0, 4.0, 4.0, 1.0, 5.0, 4.0, 3.0, 5.0, 3.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 1.0, 5.0, 1.0, 1.0, 5.0, 1.0, 1.0, 5.0, 1.0, 2.0, 5.0, 4.0, 1.0, 1.0, 3.0, 5.0, 5.0, 1.0, 5.0, 5.0, 4.0, 4.0, 1.0, 5.0, 5.0, 1.0, 4.0, 5.0, 4.0, 2.0, 5.0, 3.0, 2.0, 4.0, 2.0, 4.0, 1.0, 5.0, 1.0, 4.0, 1.0, 1.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 3.0, 4.0, 1.0, 5.0, 4.0, 2.0, 5.0, 1.0, 4.0, 1.0, 1.0, 5.0, 5.0, 5.0, 5.0, 4.0, 1.0, 3.0, 4.0, 4.0, 4.0, 1.0, 4.0, 4.0, 5.0, 3.0, 5.0, 1.0, 5.0, 1.0, 4.0, 5.0, 1.0, 1.0, 1.0, 5.0, 4.0, 5.0, 1.0, 5.0, 2.0, 5.0, 4.0, 1.0, 3.0, 5.0, 5.0, 3.0, 5.0, 4.0, 2.0, 4.0, 5.0, 5.0, 5.0, 5.0, 1.0, 4.0, 1.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 3.0, 5.0, 4.0, 3.0, 5.0, 4.0, 1.0, 5.0, 2.0, 2.0, 3.0, 4.0, 4.0, 5.0, 1.0, 4.0, 5.0, 4.0, 3.0, 2.0, 5.0, 2.0, 4.0, 1.0, 4.0, 1.0, 4.0, 1.0, 4.0, 5.0, 5.0, 3.0, 5.0, 4.0, 5.0, 4.0, 5.0, 3.0, 5.0, 4.0, 1.0, 5.0, 5.0, 1.0, 4.0, 5.0, 3.0, 5.0, 3.0, 4.0, 5.0, 4.0, 5.0, 4.0, 2.0, 2.0, 5.0, 4.0, 5.0, 3.0, 1.0,

## format + split data into train, val, and test sets

In [348]:
#SPLIT TRAIN INTO A DATASET INTO EQUAL NUMBER OF REVIEWS FOR EACH RATING. 
def train_validate_test_split_equal(df, train_percent=.6, validate_percent=.2, seed=0):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)

    rating_one = df[df['stars'] == 1.0]
    perm_one = np.random.permutation(rating_one.index)
    train_one = df.iloc[perm_one[:int(train_percent*m/5)]]

    rating_two = df[df['stars'] == 2.0]
    perm_two = np.random.permutation(rating_two.index)
    train_two = df.iloc[perm_two[:int(train_percent*m/5)]]

    rating_three = df[df['stars'] == 3.0]
    perm_three = np.random.permutation(rating_three.index)
    train_three = df.iloc[perm_three[:int(train_percent*m/5)]]

    rating_four = df[df['stars'] == 4.0]
    perm_four = np.random.permutation(rating_four.index)
    train_four = df.iloc[perm_four[:int(train_percent*m/5)]]

    rating_five = df[df['stars'] == 5.0]
    perm_five = np.random.permutation(rating_five.index)
    train_five = df.iloc[perm_five[:int(train_percent*m/5)]]

    train = pd.concat([train_one, train_two, train_three, train_four, train_five]) 


    
    leftover = pd.concat([df[int(train_percent*m/5):], df[int(train_percent*m/5):], df[int(train_percent*m/5):],df[int(train_percent*m/5):], df[int(train_percent*m/5):]]) 
  
    validate_end = int(validate_percent * m)
    validate = leftover.iloc[perm[:validate_end]]
    test = leftover.iloc[perm[validate_end:][:3000]]

    #assert train.size + validate.size + test.size == df.size

    return train, validate, test

In [349]:
# train 75% | validation 15% | test 10%
train_ratio = .50
validate_ratio = .40
test_ratio = .10
assert train_ratio + validate_ratio + test_ratio == 1

In [350]:
train_reviews, validate_reviews, test_reviews = train_validate_test_split_equal(yelp_reviews, train_ratio, validate_ratio)
# train_reviews_df, val_reviews_df, test_reviews_df = train_validate_test_split(yelp_reviews, train_ratio, validate_ratio)

In [351]:
# train_reviews, train_reviews_target, train_reviews_mask = format_reviews(xlnet_tokenizer, train_reviews_df)
# validate_reviews, test_reviews_target, validate_reviews_mask = format_reviews(xlnet_tokenizer, validate_reviews_df)
# test_reviews, test_reviews_target, _ = format_reviews(xlnet_tokenizer, test_reviews_df)

In [352]:
test_reviews


,review_id,text,stars
21416,xanFARhGRWvADHZ_ORqDnA,Kings restaurant in homar served me a\nMeatloa...,1.0
9215,R-2wEhOp87fNaYlLo1dxtA,It's been a few years since I bowled league. S...,5.0
15252,pJAUNFX0llZ3p4ep8lUKlg,Unprofessional staff what ever I asked in rega...,1.0
10305,FurylHqW9_8yRAB28LcvdA,OK.....I seriously love this place. Every Mon...,5.0
16946,rqh3bjbCDxOgtO2za1ZEaA,Just an oil change but the service was great a...,5.0
...,...,...,...
7020,TWqr4vlN4dSg7J9O9AcEZQ,It's been three weeks since the order was put ...,1.0
5618,84UDKsy3HFdAyO4Vofgpkg,Great spa. We went for the couples painting pa...,5.0
10683,LcNz9R5iqyKardATFWrn9g,"I was visiting from Boston, and needed to have...",5.0
14104,_0WPfKpw9Sg_rnSYJ7-lMA,The worst moving company ever\nThey'll give yo...,1.0


In [353]:
print(train_reviews[train_reviews['stars'] == 1.0])
print(train_reviews[train_reviews['stars'] == 2.0])
print(train_reviews[train_reviews['stars'] == 3.0])
print(train_reviews[train_reviews['stars'] == 4.0])
print(train_reviews[train_reviews['stars'] == 5.0])

                    review_id  ... stars
6288   nxhItgFpIKeDK0mMEmTBXw  ...   1.0
29532  fqBSDkaJKpxgUO1uFzBaBw  ...   1.0
6392   2LYrR24cvxATOHt3jI6Z1Q  ...   1.0
23658  _oiD516xMhzQyJSEqLLj8A  ...   1.0
12489  PXLKGZnzU0WStl-RGIp_8w  ...   1.0
...                       ...  ...   ...
1661   NdgQBCPlcOGl0DdTV1U9tQ  ...   1.0
1969   G8_cWWcnMVRRFxSWtLiQXw  ...   1.0
11828  aTbmPNg4kwTQoYRVTDsJnA  ...   1.0
16730  4f_UlOTgWo4DB9NZn4LQSg  ...   1.0
7958   Dg8Ab8pmj4MVJ9bJCVmGgg  ...   1.0

[3000 rows x 3 columns]
                    review_id  ... stars
25772  eSajrTznCoUzq1ywNFZL9g  ...   2.0
14991  wfiBBkak5cYLeOGo4E-MbA  ...   2.0
3656   s-2OBnuHFt3PEzxUu6_2rQ  ...   2.0
23331  lDSHltgMGaKoRd-DDCWs-A  ...   2.0
4401   jzmgNdIM9Rn-nSBlBEw3SA  ...   2.0
...                       ...  ...   ...
10473  HmjaZ7OEZCKdRjYNcNV4AA  ...   2.0
17368  QA7ad0HdezI96AWoPv65Cg  ...   2.0
5599   Hmn0aXBlzsDRN97SgvwBIQ  ...   2.0
21564  P8NHiX6exuj2reFmma-fMg  ...   2.0
14197  5pFdxzFsOrdj6d5g20CUjw  .

In [354]:
yelp_leftover = yelp_full[30000:]

In [355]:
train_reviews = pd.concat([train_reviews, yelp_leftover[yelp_leftover['stars'] == 2.0][:829], yelp_leftover[yelp_leftover['stars'] == 3.0][:630]])

In [356]:
print(train_reviews[train_reviews['stars'] == 1.0])
print(train_reviews[train_reviews['stars'] == 2.0])
print(train_reviews[train_reviews['stars'] == 3.0])
print(train_reviews[train_reviews['stars'] == 4.0])
print(train_reviews[train_reviews['stars'] == 5.0])

                    review_id  ... stars
6288   nxhItgFpIKeDK0mMEmTBXw  ...   1.0
29532  fqBSDkaJKpxgUO1uFzBaBw  ...   1.0
6392   2LYrR24cvxATOHt3jI6Z1Q  ...   1.0
23658  _oiD516xMhzQyJSEqLLj8A  ...   1.0
12489  PXLKGZnzU0WStl-RGIp_8w  ...   1.0
...                       ...  ...   ...
1661   NdgQBCPlcOGl0DdTV1U9tQ  ...   1.0
1969   G8_cWWcnMVRRFxSWtLiQXw  ...   1.0
11828  aTbmPNg4kwTQoYRVTDsJnA  ...   1.0
16730  4f_UlOTgWo4DB9NZn4LQSg  ...   1.0
7958   Dg8Ab8pmj4MVJ9bJCVmGgg  ...   1.0

[3000 rows x 3 columns]
                    review_id  ... stars
25772  eSajrTznCoUzq1ywNFZL9g  ...   2.0
14991  wfiBBkak5cYLeOGo4E-MbA  ...   2.0
3656   s-2OBnuHFt3PEzxUu6_2rQ  ...   2.0
23331  lDSHltgMGaKoRd-DDCWs-A  ...   2.0
4401   jzmgNdIM9Rn-nSBlBEw3SA  ...   2.0
...                       ...  ...   ...
42056  nQit5zZgH3u8ml1S9CTvZw  ...   2.0
42070  5PfoSfklPCwKLATMr0Mx3w  ...   2.0
42076  wIzgFB33R-COEnU-nqqsYg  ...   2.0
42077  aFB5SS3dV8W_JUHJNSFRsw  ...   2.0
42079  t9NHJTLBtxUNgChhIuEr5A  .

In [357]:
print(len(train_reviews.index), "yelp reviews for training")
train_reviews

15000 yelp reviews for training


,review_id,text,stars
6288,nxhItgFpIKeDK0mMEmTBXw,The lady here is a waste....I don't know why ...,1.0
29532,fqBSDkaJKpxgUO1uFzBaBw,I purchased a Cuisinart DCC-3200 coffee maker ...,1.0
6392,2LYrR24cvxATOHt3jI6Z1Q,"There pizza used to be really good, now its di...",1.0
23658,_oiD516xMhzQyJSEqLLj8A,I had the exact same experience with delivery....,1.0
12489,PXLKGZnzU0WStl-RGIp_8w,"Took 15 min for a milkshake, while I waiting b...",1.0
...,...,...,...
39325,rBI2Prwe2kr6X4uyznS0DA,So this place is tucked away in the corner and...,3.0
39347,2mLlQ9n5Ln_fykOuX_b0-A,The service is attentive. The decor attractive...,3.0
39371,ofN_d-TD1kQlsyiE6NdmDw,Pales in comparison to the Peruvian food I ate...,3.0
39372,z0KHkCFdRr7PTlSdAeTrsQ,Great place for a happy hour glass of wine. O...,3.0


In [358]:
print(len(validate_reviews.index), "yelp reviews for validation")
validate_reviews

12000 yelp reviews for validation


,review_id,text,stars
11225,st4zZZrrorOoKzc2WQ63Jg,This one is tough to rate. About 85% of the cl...,4.0
13794,twMpY9PYh6-kOasAETaRMw,I purchased the Groupon for 3 rooms as well. W...,1.0
12163,2Kp4Sf43d0eGhp8R7vlDCQ,Lisa went above and beyond my expectations to ...,5.0
29591,i7r_F2Py5W_FA497JlJqBA,I had a great experience at t mobile last nigh...,5.0
9631,tmmyCLmunJeMFTRG6iqr8Q,Queen Video is the place to be when you're loo...,4.0
...,...,...,...
22364,BYPhjwUXBLUKC-hBUtdoLQ,"I'll start off with the good, and the only rea...",2.0
6585,Fb5zwxA1kg3AHFrCr9PtuA,I absolutely love this place! So many amazing ...,5.0
4819,H2qoduHY7-mlBYyLGHZhBA,Great pulled pork sandwich!! Nice and tender t...,5.0
5060,ZBqcZZDfBpf_lRyRpdEfTw,The Toronto lsland Ferry is one of the iconic ...,5.0


In [360]:
print(len(test_reviews.index), "yelp reviews for testing")


3000 yelp reviews for testing


In [361]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

In [362]:
#experimentation 

In [363]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=200)
model.classifier.add_module('bert_activation', nn.Tanh())
model.classifier.add_module('prediction', nn.Linear(200, 5))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [364]:
model.classifier

Linear(
  in_features=768, out_features=200, bias=True
  (bert_activation): Tanh()
  (prediction): Linear(in_features=200, out_features=5, bias=True)
)

In [365]:
from transformers.data.processors.utils import InputExample


In [366]:
from transformers.data.processors.glue import glue_convert_examples_to_features

In [391]:
def get_features(df, text_col, label_col):
    l = [InputExample(guid=idx, text_a=df.loc[idx, text_col], label=df.loc[idx, label_col]) for 
       idx, row in tqdm.tqdm(df.iterrows(), total=df.shape[0])]
    features = glue_convert_examples_to_features(examples=l, 
                                    tokenizer=tokenizer,
                                    max_length=128,
                                    label_list = df[label_col].values,
                                    output_mode='classification')

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    dataset = TensorDataset(all_input_ids, all_attention_mask, all_labels)
    return dataset


In [392]:
print([i for i in test_reviews['stars'].to_list() if not isinstance(i, float)])

[]


In [393]:
test_reviews['stars'].unique()

array([1., 5., 4., 2., 3.])

In [394]:
train_reviews

,review_id,text,stars
6288,nxhItgFpIKeDK0mMEmTBXw,The lady here is a waste....I don't know why ...,1.0
29532,fqBSDkaJKpxgUO1uFzBaBw,I purchased a Cuisinart DCC-3200 coffee maker ...,1.0
6392,2LYrR24cvxATOHt3jI6Z1Q,"There pizza used to be really good, now its di...",1.0
23658,_oiD516xMhzQyJSEqLLj8A,I had the exact same experience with delivery....,1.0
12489,PXLKGZnzU0WStl-RGIp_8w,"Took 15 min for a milkshake, while I waiting b...",1.0
...,...,...,...
39325,rBI2Prwe2kr6X4uyznS0DA,So this place is tucked away in the corner and...,3.0
39347,2mLlQ9n5Ln_fykOuX_b0-A,The service is attentive. The decor attractive...,3.0
39371,ofN_d-TD1kQlsyiE6NdmDw,Pales in comparison to the Peruvian food I ate...,3.0
39372,z0KHkCFdRr7PTlSdAeTrsQ,Great place for a happy hour glass of wine. O...,3.0


In [395]:
from torch.utils.data import TensorDataset, DataLoader

In [396]:
train_reviews['stars']

6288     1.0
29532    1.0
6392     1.0
23658    1.0
12489    1.0
        ... 
39325    3.0
39347    3.0
39371    3.0
39372    3.0
39396    3.0
Name: stars, Length: 15000, dtype: float64

In [397]:
test_reviews['stars']

21416    1.0
9215     5.0
15252    1.0
10305    5.0
16946    5.0
        ... 
7020     1.0
5618     5.0
10683    5.0
14104    1.0
26445    5.0
Name: stars, Length: 3000, dtype: float64

In [398]:
train_dataset = get_features(train_reviews, 'text', 'stars')

100%|██████████| 15000/15000 [00:01<00:00, 10785.98it/s]
/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [399]:
test_dataset = get_features(test_reviews, 'text', 'stars')

100%|██████████| 3000/3000 [00:01<00:00, 1906.42it/s]
/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


TypeError: ignored

In [ ]:
train_dataset[0:9]
#the way this is structured, 3 matrices, each row of the matrix is one sentence. matrix 1) sentence tokenized 2) attention mask 3) target review. 

In [386]:
validation_dataset = get_features(validate_reviews, 'text', 'stars')

100%|██████████| 12000/12000 [00:06<00:00, 1845.10it/s]
/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


TypeError: ignored

In [ ]:
#add requires_grad for each parameter 
for param in model.classifier.parameters():
    param.requires_grad = True
model.cuda();

In [ ]:
output = model(input_ids=train_dataset[:2][0].cuda(), 
      attention_mask=train_dataset[:2][1].cuda(), 
      labels=train_dataset[:2][2].cuda());

print(output[:2])

In [ ]:
from torch.optim import Adam

In [ ]:
batch_size = 32
gradient_every = 64

accumulation_steps = gradient_every//batch_size

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(validation_dataset, batch_size=batch_size*2, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size*2, shuffle=False)

epochs = 15

lr = 0.002
optimizer = Adam(model.classifier.parameters(), lr=lr)  

In [ ]:
from tqdm import tnrange

In [ ]:
tr_losses = []
v_losses = []

for epoch in tqdm.notebook.tnrange(epochs, desc='epoch'):
    """ Training stage """
    model.train()
    epoch_tr_losses = []
    print(f'epoch {epoch+1}')
    for k, (input_ids, attention_mask, labels) in enumerate(tqdm.tqdm(train_dataloader, total=len(train_dataloader), desc='batch')):
        feed_dict = {'input_ids': input_ids.cuda(),
                     'attention_mask': attention_mask.cuda(),
                     'labels': labels.cuda()}
        
        output = model(**feed_dict)
        # gradient accumulation
        epoch_tr_losses.append(output.loss.item())
        output.loss = output.loss/accumulation_steps
        output.loss.backward()
        if (k + 1) % accumulation_steps == 0:
            optimizer.step()
            model.zero_grad()

    tr_losses.append(np.mean(epoch_tr_losses))
    print(f'train NLL loss: {np.mean(epoch_tr_losses)}')
  
    """ Validation stage """
    epoch_v_losses = [] 
    with torch.no_grad():
        for k, (input_ids, attention_mask, labels) in enumerate(tqdm.tqdm(val_dataloader, total=len(val_dataloader), desc='val batch')):
            feed_dict = {'input_ids': input_ids.cuda(),
                         'attention_mask': attention_mask.cuda(),
                         'labels': labels.cuda()} 

            output = model(**feed_dict)
            epoch_v_losses.append(output.loss.item())
        v_losses.append(np.mean(epoch_v_losses))
    print(f'validation BCE loss: {np.mean(epoch_v_losses)}')
    torch.save(model.classifier.state_dict(), str(ROOT_FOLDER / "state_dict_model.pt"))

In [ ]:
batch_predictions, batch_actual = [], []
with torch.no_grad():
    for k, (input_ids, attention_mask, labels) in enumerate(tqdm.tqdm(test_dataloader, total=len(test_dataloader), desc='test batch')):
        feed_dict = {'input_ids': input_ids.cuda(),
                     'attention_mask': attention_mask.cuda()} 
        
        pred = model(**feed_dict)[0].cpu()
        batch_predictions.append(pred.numpy())
        batch_actual.append(labels)

test batch: 100%|██████████| 16/16 [00:07<00:00,  2.10it/s]


In [ ]:
predictions = np.array([i for k in batch_predictions for i in k ])

predictions = np.argmax(predictions, axis=1)
actual = np.array([i for k in batch_actual for i in k ])

In [ ]:
test_reviews

,review_id,text,stars
4361,4xIean1TgQyrLZnkaN0wFg,Mon restaurant portugais préféré à Montréal de...,5.0
3257,B3qM3oqzS2wu6yNrYxHOlQ,I love this class! My son's development and in...,5.0
5681,hMM9tgSrp3I9eUye8SSvoA,Just took the Sunday 10:00am class with Jen. ...,4.0
6064,G2d1RaHJK-bFmpQYXzvE2w,The Twins are fabulous! I purchased my Valley ...,5.0
7459,Ly6sOTfBvRap-Vu5eHBAqg,I usually don't complain or take time to write...,1.0
...,...,...,...
9225,ICLUM9BRPBzF6qwTE6eGWQ,Our new go to place for breakfast. Nearly ever...,5.0
4859,4Spf13dGUIUJ8XQckHMwdw,Definitely the go to place for stuff you need ...,5.0
3264,762bdV36VTkmwK1Gz47RWQ,Thank you for your timely response to my reque...,4.0
9845,SWH8YiyAD7YIt5WgkgYGfg,"Our car Lurker died, and we were looking for a...",4.0


In [ ]:
test_reviews['prediction'] = predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test_reviews['distance'] = np.abs(test_reviews['stars'] - test_reviews['prediction']) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test_reviews[test_reviews['distance'] >= 3]

,review_id,text,stars,prediction,distance
3257,B3qM3oqzS2wu6yNrYxHOlQ,I love this class! My son's development and in...,5.0,1,4.0
7459,Ly6sOTfBvRap-Vu5eHBAqg,I usually don't complain or take time to write...,1.0,5,4.0
4080,JNIUh-D9EwYTdLT7B2MN9Q,I recently went to this place for cleaning my ...,5.0,1,4.0
5691,MqjBR2zz4rgyKtB-cKwZGw,Our third and final stop on the Restaurant Wee...,4.0,1,3.0
6015,SvpjSyXM2ZBcl4Z35hdSwQ,I really liked this gym when I went there. My ...,1.0,5,4.0
...,...,...,...,...,...
1207,VncR4VqbwldiJ7PKaSkKzQ,Went to the White Brick Kitchen three times in...,4.0,1,3.0
3219,34NDgFx1Ve4Eqk141j0hbQ,We regularly take our elderly mother who lives...,1.0,5,4.0
797,zc89sz0aPxs86BsfGa-qfg,Picture Homer Simpson when he sees something r...,5.0,1,4.0
755,afPyJp-v9TtRxqNCjU6rRA,"As convenience stores go, this one could be th...",1.0,5,4.0


In [ ]:
huge_differences = test_reviews[test_reviews['distance'] >= 3]

In [ ]:
#OVERPREDICTIONS
huge_differences.iloc[1]['text']

"I usually don't complain or take time to write reviews but my experience with dr Petar has been nothing but horrible.  This guy has managed to practice all the things no professional in any field should do.  Doctor-patient manners are non existent.  Doesn't keep his cellphone on silent or vibrate,  answers private calls and texts in the middle of your appointment on your dime and time, doesn't even bother to excuse himself or apologize for obvious rudeness, treats numerous patience at the same time and at any given appointment he would leave multiple times to do other things while he can't even pretend to care about your needs.  Couple times I was scheduled early in the morning and each time after a 20-30min wait I was notified by nurses that dr Petar is not even in the office yet.  When I brought it to his attention that my time is valuable too and shouldn't be scheduled at times he is still at home, shockingly he had a nerve to say go find another doctor if you don't like the servic

In [ ]:
#UNDERPREDICTIONS 
huge_differences.iloc[0]['text']

"I love this class! My son's development and interaction with others has taken off since he started a little over a month ago. Every class he does a little more than the last. Sue is a great teacher. She's great at making it fun while helping them to learn self control and patience. Someone else commented on the condition of the puppets and parachute; yeah they are a bit crusty and probably could be cleaned a little better, but I decided to put my germaphobe fears aside for my child's development. Even if he gets a cold or two from class (which he hasn't after 5 weeks) that seems to be par for the course when it comes to interacting with other children and the benefits to his development outweigh a few sniffles in my opinion!"

In [ ]:
huge_differences.iloc[2]['text']

"I recently went to this place for cleaning my car, after one of my Uber passengers throw up on the back seat. Since Uber was not kind enough to cover the fees entirely, I asked Evan if we can clean the car with the amount that Uber will cover, and he was really generous to accept, giving a signification discount to me so I can clean my car with the amount Uber provided. The service was great. They cleaned the whole interior and treated the cabin with ozone, so the smell completely disappeared (note that vomit has a terrible odour!). \n\nI never saw my car so clean, since I bought it used, and even the time I purchased the car from the dealership, it wasn't as clean as I got the car from this shop. \n\nAbout what they did, they washed the seats with shampoo and by vacuum machine sucked the whole water out, so I didn't feel the seats being wet. They also cleaned my dashboard and all the plastic components in the car and I beloved waxed them as well. \n\nOverall, very happy with their se

#Model

## model construction

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, rnn_size, vader_size, num_layers=1, dropout=0, use_vader=True):
        super().__init__()
        
        #################
        #    INPUT 1    #
        #################
        # Create an embedding layer, with 768 hidden layers
        self.xlnet = torch.hub.load('huggingface/pytorch-transformers', 'model', 'xlnet-base-cased')
        for param in self.xlnet.layer.parameters():
          param.requires_grad = False

          
        # Output: (vocab_size x 768), where 768 hidden layers of XLNet

        # Coming in: torch.Size([BATCH_SIZE, vocab_size, 768])
        #   (XLNet has 768 hidden layers, https://huggingface.co/transformers/pretrained_models.html)
        conv2d_c_in = 1
        conv2d_c_out = 1
        conv2d_kernel_W = 5 # along Embedding Length
        conv2d_kernel_H = 5 # along Word Length

        self.conv2D_layer = nn.Conv2d(conv2d_c_in, conv2d_c_out, (conv2d_kernel_H, conv2d_kernel_W))
        # Filter of (conv2d_kernel_H, conv2d_kernel_W), Cin = 1, Cout = 1

        # Output:
        conv2d_out_Hout = vocab_size - ((conv2d_kernel_H - 1) // 2) * 2 # Vocab Size
        conv2d_out_Wout = 768 - ((conv2d_kernel_W - 1) // 2) * 2        # length

        self.max_pool_2d = nn.MaxPool2d((conv2d_out_Hout, 1))
        max_pool_2d_out_height = conv2d_out_Hout // conv2d_out_Hout
        max_pool_2d_out_length = conv2d_out_Wout // 1
        #################
        #  INPUT 1 END  #
        #################
        
        #################
        #    INPUT 2    #
        #################
        self.lstm = None
        if use_vader:
          self.lstm = nn.LSTM(input_size=1, hidden_size=1, num_layers=num_layers, batch_first=True, dropout=dropout)
        else:
          vader_size = 0
        #################
        #  INPUT 2 END  #
        #################

        self.dropout = nn.Dropout(dropout)
        # print(max_pool_2d_out_length + vader_size)

        hidden_layer_dense = 100

        self.dense = nn.Sequential(
                nn.Linear(max_pool_2d_out_length + vader_size, hidden_layer_dense),
                nn.ReLU()
            )
        self.output = nn.Linear(hidden_layer_dense, 5) # classify yelp_reviews into 5 ratings
    
    def forward_input_vectorized(self, x):
      xlnet_out = self.xlnet(x)
      xlnet_out_hidden = xlnet_out.last_hidden_state
      batches_len, word_len, embedding_len = xlnet_out_hidden.shape
      xlnet_out_hidden = xlnet_out_hidden.reshape(batches_len, 1, word_len, embedding_len)
      conv2d_out = self.conv2D_layer(xlnet_out_hidden)
      result = self.max_pool_2d(conv2d_out)
      # print(result.shape)
      result = result.squeeze(1).squeeze(1)
      return result

    def forward_input_vader(self, x):
      batch_size, vader_len = x.shape
      # print(x.reshape(batch_size, vader_len, 1).shape)
      output, _ = self.lstm(x.reshape(batch_size, vader_len, 1))
      # print(output.shape)
      output = output.squeeze(2)
      return output

    def forward(self, vectorized_words, vader):
        input1 = self.forward_input_vectorized(vectorized_words)

        if self.lstm:
          input2 = self.forward_input_vader(vader)
          combined_input = (input1, input2)
        else:
          combined_input = (input1,) # Tuples need the stray comma

        # print(input1.size(), input2.size())

        combined_input = torch.cat(combined_input, dim=1)

        lstm_drop = self.dropout(combined_input)
        logits = self.dense(lstm_drop)
        logits = self.output(logits)
        return logits
    
    def loss_fn(self, prediction, target):
      loss_criterion = nn.CrossEntropyLoss(reduction='none')
      return torch.mean(loss_criterion(prediction, target))

In [ ]:
model = LanguageModel(vocab_size=MAX_LEN, rnn_size=256, vader_size=MAX_LEN_VADER)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


## train the model

In [ ]:
# num_of_validaion_set = 20 #len(validate_reviews)

# batch_val = format_reviews(xlnet_tokenizer, validate_reviews, range(num_of_validaion_set), review_sentiment_dict=review_sentiment_dict) # This cell may take a while

# (batch_input_val, batch_target_val, batch_review_sentiment_val, batch_target_mask_val) = batch_val

In [ ]:
def run_validation(model, use_all=False, mode="val"):
  reviews_dataset = None
  if mode == "val":
    print("Running Validation")
    mode = "Validation"
    reviews_dataset = validate_reviews
  elif mode == "test":
    print("Running Testing")
    mode = "Test"
    reviews_dataset = test_reviews
  else:
    assert False, "Invalid mode"
  num_of_review_set = len(reviews_dataset) if use_all else 1000
  indices = np.random.permutation(len(reviews_dataset))
  t = tqdm.notebook.tqdm(range(0, ( num_of_review_set // BATCH_SIZE) + ( 1 if num_of_review_set % BATCH_SIZE > 0 else 0 )))
  loss_val_total = 0
  accuracy_val_total = 0
  temp_count = 0
  for i in t:
    val_start_i = i*BATCH_SIZE
    val_end_i = (i+1)*BATCH_SIZE
    # print(val_start_i, val_end_i, indices.shape)
    batch_val = format_reviews(xlnet_tokenizer, reviews_dataset, indices[val_start_i:val_end_i], review_sentiment_dict=review_sentiment_dict)
    (batch_input_val, batch_target_val, batch_review_sentiment_val, batch_target_mask_val) = batch_val
    # print(batch_input_val.shape, batch_review_sentiment_val.shape)
    (batch_input_val, batch_target_val) = list_to_device((batch_input_val, batch_target_val))
    batch_target_mask_val, batch_review_sentiment_val = list_to_device((batch_target_mask_val, batch_review_sentiment_val))
    # print(batch_input_val.shape, batch_review_sentiment_val.shape)
    prediction_val = model.forward(batch_input_val, batch_review_sentiment_val)
    # print(prediction_val.size(), batch_target_val.size())
    # print(prediction_val, batch_target_val)
    loss_val_total += model.loss_fn(prediction_val, batch_target_val).item()
    # print(loss_val)
    accuracy_val_total += torch.mean(torch.eq(prediction_val.argmax(dim=1,keepdim=False),batch_target_val).float()).item()
    temp_count += 1
    if i % round(8000 / BATCH_SIZE) == 0 and i != 0 and use_all:
      print(mode, "Prelim Evaluation set loss:", loss_val_total / temp_count, mode, "Prelim Accuracy:", accuracy_val_total / temp_count)
  loss_val = loss_val_total / temp_count
  accuracy_val = accuracy_val_total / temp_count
  print(mode, "Evaluation set loss:", loss_val, mode, "Accuracy set %:", accuracy_val)

In [ ]:
losses = []
accuracies = []

epoch_start = 0
t_start = 0

In [ ]:
# ONLY RUN THIS CELL (and next cell) if want to load checkpoint
# If you accidentally run this cell, no harm done (be careful with next cell!!!)

checkpoint = None
try:
  checkpoint = torch.load(str(TORCH_CHECKPOINT_MODEL))
  print("Checkpoint loaded")
except:
  print("No Checkpoint loaded")

No Checkpoint loaded


In [ ]:
lr = 1e-4
optimizer_method = optim.Adam
optimizer = optimizer_method(model.parameters(), lr=lr)

In [ ]:
# ONLY RUN THIS CELL if want to load checkpoint

if checkpoint:
  epoch_start = checkpoint['epoch']
  t_start = checkpoint['t']
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer = optimizer_method(model.parameters(), lr=lr)
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  losses = checkpoint['losses']
  accuracies = checkpoint['accuracies']
  model.to(device)

  print("Checkpoint")
  run_validation(model)

  print(f"Checkpoint Epoch: {epoch_start} Iteration: {i} Loss: {np.mean(losses[-10:])} Accuracy: {np.mean(accuracies[-10:])}")

In [ ]:
# set model to training mode
# Needs to be placed after the Checkpoint file loading
model.train()

LanguageModel(
  (xlnet): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

DATASET = train_reviews

# Constants of interest: BATCH_SIZE, EPOCHS

since = time.time()

# start training
for epoch in range(epoch_start, EPOCHS):
  indices = np.random.permutation(DATASET.shape[0])

  dataset_batch_cap = ( DATASET.shape[0] // BATCH_SIZE ) + (1 if DATASET.shape[0] % BATCH_SIZE > 0 else 0)

  t = tqdm.notebook.tqdm(range(t_start, dataset_batch_cap), initial = t_start, total = dataset_batch_cap)
  
  for i in t:
    # batch
    batch = format_reviews(xlnet_tokenizer, DATASET, indices[i*BATCH_SIZE:(i+1)*BATCH_SIZE], review_sentiment_dict=review_sentiment_dict)
    (batch_input, batch_target, batch_review_sentiment, batch_target_mask) = batch
    # for item in (batch_input, batch_target, batch_review_sentiment, batch_target_mask):
    #   print(item.size())
    (batch_input, batch_target, batch_target_mask, batch_review_sentiment) = list_to_device((batch_input, batch_target, batch_target_mask, batch_review_sentiment))
    model.to(device)
    
    # forward pass
    prediction = model.forward(batch_input, batch_review_sentiment)
    # print(prediction.size(), batch_target.size())
    loss = model.loss_fn(prediction, batch_target)
    # print(loss)
    losses.append(loss.item())
    accuracy = torch.mean(torch.eq(prediction.argmax(dim=1,keepdim=False),batch_target).float())
    accuracies.append(accuracy.item())
    
    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # visuallize data
    if i % 1000 == 0 and i != t_start:
      torch.save({'epoch': epoch,
                  't': i,
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'losses': losses,
                  'accuracies': accuracies
                  }, str(TORCH_CHECKPOINT_MODEL))
      run_validation(model)
      print(f"Epoch: {epoch} Iteration: {i} Train Loss: {np.mean(losses[-10:])} Train Accuracy: {np.mean(accuracies[-10:])}")

  t_start = 0


RuntimeError: ignored

In [ ]:
# Save the latest model
print("Saving latest model to", str(TORCH_CHECKPOINT_MODEL))
torch.save({'epoch': EPOCHS,
            't': (DATASET.shape[0] // BATCH_SIZE)+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'losses': losses,
            'accuracies': accuracies
            }, str(TORCH_CHECKPOINT_MODEL))

## evaluate model

In [ ]:
# set model to evaluation model
model.eval()

In [ ]:
run_validation(model, True, "val")

In [ ]:
run_validation(model, True, "test")

Running Testing


Test Prelim Evaluation set loss: 0.7104197494507193 Test Prelim Accuracy: 0.7723303393213573
Test Prelim Evaluation set loss: 0.7166598277030648 Test Prelim Accuracy: 0.7687937062937062
Test Prelim Evaluation set loss: 0.7128958379717011 Test Prelim Accuracy: 0.7719020652898068
Test Prelim Evaluation set loss: 0.714400890948384 Test Prelim Accuracy: 0.7722701149425287
Test Prelim Evaluation set loss: 0.7153426486878145 Test Prelim Accuracy: 0.7720661735305878
Test Prelim Evaluation set loss: 0.7138283398115349 Test Prelim Accuracy: 0.7721176274575141

Test Evaluation set loss: 0.04473292892023305 Test Accuracy set %: 0.0482227303123038


#Playground

In [ ]:
# This is a cheap solution to stops any run all that reaches my Playground
hard_stop = input("Hard Stop here. Enter any key to allow passage.")

if len(hard_stop) == 0:
  raise Exception("Hard Stop")

In [ ]:
print(list(tokenize(STARTER["text"]))[:3])

In [ ]:
import urllib.request
import io
import sentencepiece as spm

# https://github.com/google/sentencepiece/tree/master/python

# Loads model from URL as iterator and stores the model to BytesIO.
model = io.BytesIO()
spm.SentencePieceTrainer.train(
      sentence_iterator=STARTER["text"], model_writer=model, vocab_size=1000)

# Serialize the model as file.
# with open('out.model', 'wb') as f:
#   f.write(model.getvalue())

# Directly load the model from serialized model.
sp = spm.SentencePieceProcessor(model_proto=model.getvalue())
print(sp.encode('this is test'))